# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction

In this [Kaggle competition](https://www.kaggle.com/c/stanford-covid-vaccine/overview) we try to develop models and design rules for RNA degradation. As the overview of the competition states:

>mRNA vaccines have taken the lead as the fastest vaccine candidates for COVID-19, but currently, they face key potential limitations. One of the biggest challenges right now is how to design super stable messenger RNA molecules (mRNA). Conventional vaccines (like your seasonal flu shots) are packaged in disposable syringes and shipped under refrigeration around the world, but that is not currently possible for mRNA vaccines.
>
>Researchers have observed that RNA molecules have the tendency to spontaneously degrade. This is a serious limitation--a single cut can render the mRNA vaccine useless. Currently, little is known on the details of where in the backbone of a given RNA is most prone to being affected. Without this knowledge, current mRNA vaccines against COVID-19 must be prepared and shipped under intense refrigeration, and are unlikely to reach more than a tiny fraction of human beings on the planet unless they can be stabilized.

<img src="images/banner.png" width="1000" style="margin-left: auto; margin-right: auto;"> 

The model should predict likely degradation rates at each base of an RNA molecule. The training data set is comprised of over 3000 RNA molecules and their degradation rates at each position.

# Install necessary packages

We can install the necessary package by either running `pip install --user <package_name>` or include everything in a `requirements.txt` file and run `pip install --user -r requirements.txt`. We have put the dependencies in a `requirements.txt` file so we will use the former method.

> NOTE: Do not forget to use the `--user` argument. It is necessary if you want to use Kale to transform this notebook into a Kubeflow pipeline

In [1]:
!pip install --user -r requirements.txt

     |████████████████████████████████| 320.4 MB 20 kB/s s eta 0:00:01
     |████████████████████████████████| 42 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 20.1 MB 66.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 34.4 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 59.7 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 20.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 10.6 MB 45.3 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.3 MB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 68.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 51.5 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consi

# Imports

In this section we import the packages we need for this example. Make it a habit to gather your imports in a single place. It will make your life easier if you are going to transform this notebook into a Kubeflow pipeline using Kale.

In [2]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer

# Project hyper-parameters

In this cell, we define the different hyper-parameters. Defining them in one place makes it easier to experiment with their values and also facilitates the execution of HP Tuning experiments using Kale and Katib.

In [3]:
# Hyper-parameters
LR = 1e-3
EPOCHS = 10
BATCH_SIZE = 64
EMBED_DIM = 100
HIDDEN_DIM = 128
DROPOUT = .5
SP_DROPOUT = .3
TRAIN_SEQUENCE_LENGTH = 107

Set random seed for reproducibility and ignore warning messages.

In [4]:
tf.random.set_seed(42)
np.random.seed(42)

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# Load and preprocess data

In this section, we load and process the dataset to get it in a ready-to-use form by the model. First, let us load and analyze the data.

## Load data

The data are in `json` format, thus, we use the handy `read_json` pandas method. There is one train data set and two test sets (one public and one private).

In [5]:
train_df = pd.read_json("data/train.json", lines=True)
test_df = pd.read_json("data/test.json", lines=True)

We also load the `sample_submission.csv` file, which will prove handy when we will be creating our submission to the competition.

In [6]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")

Let us now explore the data, their dimensions and what each column mean. To this end, we use the pandas `head` method to visualize a small sample (five rows by default) of our data set.

In [7]:
train_df.head()

,SN_filter,deg_50C,deg_Mg_50C,deg_Mg_pH10,deg_error_50C,deg_error_Mg_50C,deg_error_Mg_pH10,deg_error_pH10,deg_pH10,id,index,predicted_loop_type,reactivity,reactivity_error,seq_length,seq_scored,sequence,signal_to_noise,structure
0,1,"[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...",id_001f94081,0,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,"[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...",107,68,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,6.894,.....((((((.......)))).)).((.....((..((((((......
1,0,"[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",id_0049f53ba,1,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,"[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...",107,68,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,0.193,.....(((((((((((((((((((((((....)))))))))).)))...
2,1,"[0.9501000000000001, 1.7974999999999999, 1.499...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...",id_006f36f57,2,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,"[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.0931, 0.13290000000000002, 0.11280000000000...",107,68,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,8.800,.....((((.((.....((((.(((.....)))..((((......)...
3,0,"[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[121286.7181, 121286.7182, 121286.7181, 121286...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",id_0082d463b,3,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,"[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...",107,68,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,0.104,......((((((((((((((((......))))))))))))))))((...
4,0,"[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...",id_0087940f4,4,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,"[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...",107,68,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,0.423,.....(((((((.((((((((((((.(((((((((....)))))))...


We see a lot of strange entries, so, let us try to see what they are:

* `sequence`: An 107 characters long string in Train and Public Test (130 in Private Test), which describes the RNA sequence, a combination of A, G, U, and C for each sample.
* `structure`: An 107 characters long string in Train and Public Test (130 in Private Test), which is a combination of `(`, `)`, and `.` characters that describe whether a base is estimated to be paired or unpaired. Paired bases are denoted by opening and closing parentheses (e.g. (....) means that base 0 is paired to base 5, and bases 1-4 are unpaired).
* `predicted_loop_type`: An 107 characters long string, which describes the structural context (also referred to as 'loop type') of each character in sequence. Loop types assigned by bpRNA from Vienna RNAfold 2 structure. From the bpRNA_documentation: `S`: paired "Stem" `M`: Multiloop `I`: Internal loop `B`: Bulge `H`: Hairpin loop `E`: dangling End `X`: eXternal loop.

Then, we have `signal_to_noise`, which is quality control feature. It records the measurements relative to their errors; the higher value the more confident measurements are.

The `*_error_*` columns calculate the errors in experimental values obtained in corresponding `reactivity` and `deg_*` columns.

The last five columns (i.e., `recreativity` and `deg_*`) are out depended variables, our targets. Thus, for every base in the molecule we should predict five different values.

These are the main columns we care about. For more details, visit the competition [info](https://www.kaggle.com/c/stanford-covid-vaccine/data).

## Preprocess data

We are now ready to preprocess the data set. First, we define the symbols that encode certain features (e.g. the base symbol or the structure), the features and the target variables.

In [8]:
symbols = "().ACGUBEHIMSX"
feat_cols = ["sequence", "structure", "predicted_loop_type"]
target_cols = ["reactivity", "deg_Mg_pH10", "deg_Mg_50C", "deg_pH10", "deg_50C"]
error_cols = ["reactivity_error", "deg_error_Mg_pH10", "deg_error_Mg_50C", "deg_error_pH10", "deg_error_50C"]

In order to encode values like strings or characters and feed them to the neural network, we need to tokenize them. The `Tokenizer` class will assign a number to each character.

In [9]:
tokenizer = Tokenizer(char_level=True, filters="")
tokenizer.fit_on_texts(symbols)

Moreover, the tokenizer keeps a dictionary, `word_index`, from which we can get the number of elements in our vocabulary. In this case, we only have a few elements, but if our dataset was a whole book, that function would be handy.

> NOTE: We should add `1` to the length of the `word_index` dictionary to get the correct number of elements.

In [10]:
# get the number of elements in the vocabulary
vocab_size = len(tokenizer.word_index) + 1

We are now ready to process our features. First, we transform each character sequence (i.e., `sequence`, `structure`, `predicted_loop_type`) into number sequences and concatenate them together. The resulting shape should be `(num_examples, 107, 3)`.

> Now, we should do this in a way that would permit us to use this processing function with KFServing. Thus, since Numpy arrays are not JSON serializable, this function should accept and return pure Python lists.

In [11]:
def process_features(example):
    sequence_sentences = example[0]
    structure_sentences = example[1]
    loop_sentences = example[2]
    
    # transform character sequences into number sequences
    sequence_tokens = np.array(
        tokenizer.texts_to_sequences(sequence_sentences)
    )
    structure_tokens = np.array(
        tokenizer.texts_to_sequences(structure_sentences)
    )
    loop_tokens = np.array(
        tokenizer.texts_to_sequences(loop_sentences)
    )
    
    # concatenate the tokenized sequences
    sequences = np.stack(
        (sequence_tokens, structure_tokens, loop_tokens),
        axis=1
    )
    sequences = np.transpose(sequences, (2, 0, 1))
    
    prepared = sequences.tolist()
    
    return prepared[0]

In the same way we process the labels. We should just extract them and transform them into the correct shape. The resulting shape should be `(num_examples, 68, 5)`.

In [12]:
def process_labels(df):
    df = df.copy()
    
    labels = np.array(df[target_cols].values.tolist())
    labels = np.transpose(labels, (0, 2, 1))
    
    return labels

In [13]:
public_test_df = test_df.query("seq_length == 107")
private_test_df = test_df.query("seq_length == 130")

We are now ready to process the data set and make the features ready to be consumed by the model.

In [14]:
x_train = [process_features(row.tolist()) for _, row in train_df[feat_cols].iterrows()]
y_train = process_labels(train_df)

unprocessed_x_public_test = [row.tolist() for _, row in public_test_df[feat_cols].iterrows()]
unprocessed_x_private_test = [row.tolist() for _, row in private_test_df[feat_cols].iterrows()]

# Define and train the model

We are now ready to define our model. We have to do with sequences, thus, it makes sense to use RNNs. More specifically, we will use bidirectional Gated Recurrent Units (GRUs) and Long Short Term Memory cells (LSTM). The output layer shoud produce 5 numbers, so we can see this as a regression problem.

First let us define two helper functions for GRUs and LSTMs and then, define the body of the full model.

In [15]:
def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
         tf.keras.layers.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [16]:
def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

The model has an embedding layer. The embedding layer projects the tokenized categorical input into a high-dimensional latent space. For this example we treat the dimensionality of the embedding space as a hyper-parameter that we can use to fine-tune the model.

In [17]:
def build_model(vocab_size, seq_length=TRAIN_SEQUENCE_LENGTH, pred_len=68,
                embed_dim=EMBED_DIM,
                hidden_dim=HIDDEN_DIM, dropout=DROPOUT, sp_dropout=SP_DROPOUT):
    inputs = tf.keras.layers.Input(shape=(seq_length, 3))

    embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    
    hidden = tf.keras.layers.SpatialDropout1D(sp_dropout)(reshaped)
    
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation="linear")(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    return model

In [18]:
model = build_model(vocab_size)

In [19]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 107, 3)]          0         
_________________________________________________________________
embedding (Embedding)        (None, 107, 3, 100)       1500      
_________________________________________________________________
tf_op_layer_Reshape (TensorF [(None, 107, 300)]        0         
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 107, 300)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 107, 256)          330240    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 107, 256)          394240    
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 68, 256)]        

Submissions are scored using MCRMSE (mean columnwise root mean squared error):

<img src="images/mcrmse.png" width="250" style="margin-left: auto; margin-right: auto;">

Thus, we should code this metric and use it as our objective (loss) function.

In [20]:
class MeanColumnwiseRMSE(tf.keras.losses.Loss):
    def __init__(self, name='MeanColumnwiseRMSE'):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

We are now ready to compile and fit the model.

In [21]:
model.compile(tf.optimizers.Adam(learning_rate=LR), loss=MeanColumnwiseRMSE())

In [22]:
history = model.fit(np.array(x_train), np.array(y_train), 
                    validation_split=.1, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/10
34/34 [==============================] - 19s 557ms/step - loss: 0.6024 - val_loss: 0.6217
Epoch 2/10
34/34 [==============================] - 17s 491ms/step - loss: 0.5362 - val_loss: 0.5990
Epoch 3/10
34/34 [==============================] - 17s 491ms/step - loss: 0.5134 - val_loss: 0.5856
Epoch 4/10
34/34 [==============================] - 17s 489ms/step - loss: 0.4990 - val_loss: 0.5706
Epoch 5/10
34/34 [==============================] - 17s 491ms/step - loss: 0.4914 - val_loss: 0.5653
Epoch 6/10
34/34 [==============================] - 16s 484ms/step - loss: 0.4848 - val_loss: 0.5566
Epoch 7/10
34/34 [==============================] - 17s 486ms/step - loss: 0.4768 - val_loss: 0.5508
Epoch 8/10
34/34 [==============================] - 17s 488ms/step - loss: 0.4709 - val_loss: 0.5452
Epoch 9/10
34/34 [==============================] - 17s 489ms/step - loss: 0.4650 - val_loss: 0.5404
Epoch 10/10
34/34 [==============================] - 17s 488ms/step - loss: 0.4605 - val_lo

In [23]:
validation_loss = history.history.get("val_loss")[0]

## Evaluate the model

Finally, we are ready to evaluate the model using the two test sets.

In [24]:
model_public = build_model(vocab_size, seq_length=107, pred_len=107)
model_private = build_model(vocab_size, seq_length=130, pred_len=130)

model_public.set_weights(model.get_weights())
model_private.set_weights(model.get_weights())

In [25]:
public_preds = model_public.predict(np.array([process_features(x) for x in unprocessed_x_public_test]))
private_preds = model_private.predict(np.array([process_features(x) for x in unprocessed_x_private_test]))

# Submission

Last but note least, we create our submission to the Kaggle competition. The submission is just a `csv` file with the specified columns.

In [26]:
preds_ls = []

for df, preds in [(public_test_df, public_preds), (private_test_df, private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=target_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_ls.append(single_df)

preds_df = pd.concat(preds_ls)
preds_df.head()

,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C,id_seqpos
0,0.748846,0.733755,0.648332,2.133584,0.779026,id_00073f8be_0
1,2.066984,2.999857,2.974135,3.784941,2.447127,id_00073f8be_1
2,1.254770,0.605719,0.711068,0.715006,0.806189,id_00073f8be_2
3,1.462632,1.224108,1.659209,1.286412,1.373700,id_00073f8be_3
4,0.945727,0.716441,0.877251,0.817625,0.899158,id_00073f8be_4


In [27]:
submission = sample_submission_df[['id_seqpos']].merge(preds_df, on=['id_seqpos'])
submission.to_csv('submission.csv', index=False)